In [1]:
import csv
from bs4 import BeautifulSoup

In [8]:
with open(r"D:\download\1688.html", "r", encoding="gb18030", errors="replace") as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')

data = []
listBox = soup.find('div', id='listBox')
orders = listBox.find_all('li', class_='order-item') if listBox else []

for order in orders:
    title = order.find('div', class_='order-title')

    #retrive order number
    order_id_span = title.find("span", class_="order-id") if title else None
    order_number = order_id_span.text.replace("订单号：", "").strip() if order_id_span else "No Order ID"

    #retrieve order date
    date_span = title.find('span', class_='date')
    date = date_span.text.strip() if date_span else 'No date'

    #retrieve supplier's detail
    supplier_tag = title.find('a', class_='bannerCorp')

    #retrieve the supplier name
    supplier_name = supplier_tag['data-copytitle'] if supplier_tag else 'No name'

    #retrieve supplier's link
    supplier_link = supplier_tag['href'] if supplier_tag else 'No link'

    # retrieve detaild of each item
    order_detail = order.find('div', class_='order-detail')

    
    #retrieve total paid
    total_td = order_detail.find('td', class_='s6')

    # without discount
    total_no_discount_tag = total_td.find('div', class_='sum-payment-original text-lesser') if total_td else None
    total_no_discount = total_no_discount_tag.text.strip() if total_no_discount_tag else 'No total'

    # with discount
    total_w_discount_tag = total_td.find('div', class_='total') if total_td else None
    total_w_discount = total_w_discount_tag.text.strip() if total_w_discount_tag else 'No total'

    # Extract item in the order
    items = order_detail.find_all('tr', style=True) if order_detail else []

    for item in items:
        # retrieve image_url
        image_td = item.find('td', class_='s1')
        image_tag = image_td.find('img') if image_td else None
        image = 'D:/download' + image_tag['src'] if image_tag else 'No image'

        #extract product link
        product_td = item.find("td", class_="s2")
        product_link_tag = product_td.find("a", class_="productName") if product_td else None
        product_link = product_link_tag["href"] if product_link_tag else "No Link"

        #retrieve price
        price_td = item.find('td', class_='s3')
        price_tag = price_td.find('div', {"data-unit-price": "isbuyer"}) if price_td else None
        price = price_tag.text.strip() if price_tag else 'No price'

        #retrieve quantity
        quantity_td = item.find('td', class_='s4')
        quantity = quantity_td.text.strip() if quantity_td else 'No quantity'

        data.append([order_number, date, supplier_name, supplier_link, image, product_link, price, quantity, total_no_discount, total_w_discount])


with open("all_orders.csv", "w", newline="", encoding="utf-8-sig") as file:
    writer = csv.writer(file)
    writer.writerow(['order_id', 'date', 'supplier', 'supplier_link', 'image', 'product_link', 'price', 'quantity', 'total_before_discount', 'total_after_discount'])
    writer.writerows(data)

print("✅ CSV file created successfully!")


✅ CSV file created successfully!
